## libraries

In [ ]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm

## data load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = 'your_path'
review = pd.read_csv(file_paht+'file_name.csv')            # 샘플링된 review 데이터
with open(file_path+'/sentences_sample.pkl', 'rb') as f :  # review에서 분리했던 문장 딕셔너리
  rev_sentences = pickle.load(f)

## Extract Keywords by KeyBERT

In [ ]:
!pip install keybert
from keybert import KeyBERT

In [ ]:
# 키워드 추출 모델 load
kw_model = KeyBERT()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 키워드 담을 딕셔너리
keywords_dict={}

In [ ]:
keys = list(rev_sentences.keys())  # review_id

start, end = 0, len(rev_sentences)
length = end-start

# tqdm 설정
with tqdm(total=length) as pbar :
  for key in keys[:end] :
    if key in keywords_dict :
      pbar.update(1)
      continue
    else : # 이어서 담기
      temp_one = []
      temp_two = []
      for sen in rev_sentences[key] :
        kws_one = kw_model.extract_keywords(sen, keyphrase_ngram_range=(1,1), stop_words='english', top_n=1) # 1-gram
        kws_two = kw_model.extract_keywords(sen, keyphrase_ngram_range=(1,2), stop_words='english', top_n=1) # 2-gram
        kw_one = kws_one[0][0] if kws_one else ''
        kw_two = kws_two[0][0] if kws_two else ''
        temp_one.append(kw_one)  # 문장별 키워드 추가
        temp_two.append(kw_two)
      keywords_dict[key] = [temp_one, temp_two]
      pbar.update(1)
    # 1000개 단위로 pickle 저장
    if len(keywords_dict)%1000 == 0 :
      with open(file_path+'/keywords_sample.pkl', 'wb') as f :
        pickle.dump(keywords_dict, f)
  with open(file_path+'/keywords_sample.pkl', 'wb') as f :
    pickle.dump(keywords_dict, f)

  0%|          | 0/30880 [00:00<?, ?it/s]

In [ ]:
# 필요 시 load하여 확인
with open(file_path+'/keywords_sample.pkl', 'rb') as f :
  keywords_dict = pickle.load(f)

## Add Keywords Columns

In [ ]:
review_sample = pd.read_csv(file_path+'file_name.csv')               # csv로 load 시
# review_sample = pd.read_json(file_path+'file_name.json', lines=True) # json으로 load 시

In [ ]:
keywords_dict

{1443659: [['uber', 'wait'], ['deli uber', 'check person']],
 1044648: [['dinner',
   'brunch',
   'nachos',
   'entree',
   'dishes',
   'brulee',
   'mojito',
   'birthdays',
   'feel',
   'crowded',
   'parking'],
  ['dinner good',
   'brunch great',
   'asian nachos',
   'entree',
   'hh dishes',
   'creme brulee',
   'blueberry mojito',
   'place celebrate',
   'nice feel',
   'bar area',
   'parking sux']],
 5693786: [['restaurant', 'business', 'lunch', 'open'],
  ['logo restaurant', 'business', 'menu breakfast', 'open 24']],
 1166589: [['slooooww', 'food', 'group', 'group', 'servers', 'felt', 'drinks'],
  ['going slooooww',
   'food okay',
   'group slow',
   'came group',
   'servers watched',
   'felt bad',
   'running food']],
 6678286: [['deli', 'philly', 'tasty', 'chips', 'hungry', 'yuge'],
  ['deli', 'philly club', 'tasty good', 'bags chips', 'come hungry', 'yuge']],
 2462810: [['il', 'app', '8pm', 'cheddars'],
  ['worst service', 'order waited', 'cleaning 8pm', 'going che

In [ ]:
# 리스트 생성
unigram = []
biagram = []

# 딕셔너리에서 unigram, bigram 각각 추출하여 리스트에 추가
for key in keywords_dict:
  unigram.append(keywords_dict[key][0])
  biagram.append(keywords_dict[key][1])
unigram

[['uber', 'wait'],
 ['dinner',
  'brunch',
  'nachos',
  'entree',
  'dishes',
  'brulee',
  'mojito',
  'birthdays',
  'feel',
  'crowded',
  'parking'],
 ['restaurant', 'business', 'lunch', 'open'],
 ['slooooww', 'food', 'group', 'group', 'servers', 'felt', 'drinks'],
 ['deli', 'philly', 'tasty', 'chips', 'hungry', 'yuge'],
 ['il', 'app', '8pm', 'cheddars'],
 ['steak',
  'grill',
  'beef',
  'marmont',
  'steaks',
  'trimmed',
  'meat',
  'fries',
  'beef',
  'service',
  'decor',
  'floor',
  'dive',
  'tom',
  'warm',
  'don',
  'make',
  '5k',
  'doing',
  'decorate',
  'steaks',
  'steak'],
 ['food', 'taco', 'recycling', 'guero'],
 ['food', 'hockey', 'waitress', 'water', 'appetizer'],
 ['edmonton', 'food', 'gnocchi', 'plating', 'lobster'],
 ['taco', 'tacos', 'queso', 'recommend'],
 ['indiana',
  'indiana',
  'diner',
  'neighborhood',
  'seat',
  'prices',
  'meals',
  'meals',
  'toast',
  'bowl',
  'steak',
  'pancakes',
  'waitress',
  'waitress',
  'service',
  'restaurant'],

In [ ]:
# 컬럼 추가
review_sample['keywords_ug'] = unigram
review_sample['keywords_bg'] = biagram

In [ ]:
review_sample

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,sentences,sentence_counts,keywords_ug,keywords_bg
index,,,,,,,,,,,,,
184,1443659,607521,108373,5.0,0,0,0,Found 51st Deli on Uber Eats and we loved it!!...,2017-01-14 21:23:10,['Found 51st Deli on Uber Eats and we loved it...,2,"[uber, wait]","[deli uber, check person]"
207,1044648,182108,134961,5.0,0,0,0,Love love love this place for HH n dinner is g...,2017-08-07 23:48:51,['Love love love this place for HH n dinner is...,11,"[dinner, brunch, nachos, entree, dishes, brule...","[dinner good, brunch great, asian nachos, entr..."
237,5693786,749220,54049,5.0,0,0,0,Not sure the significance behind the name and ...,2018-09-25 14:49:55,['Not sure the significance behind the name an...,4,"[restaurant, business, lunch, open]","[logo restaurant, business, menu breakfast, op..."
277,1166589,572170,9263,2.0,1,0,0,The service is going to be slooooww. And the f...,2017-10-02 01:52:47,"['The service is going to be slooooww.', 'And ...",7,"[slooooww, food, group, group, servers, felt, ...","[going slooooww, food okay, group slow, came g..."
323,6678286,816928,108373,5.0,0,0,0,Much more than a deli! Wife and I had the Phil...,2017-12-29 20:18:59,"['Much more than a deli!', 'Wife and I had the...",6,"[deli, philly, tasty, chips, hungry, yuge]","[deli, philly club, tasty good, bags chips, co..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2679757,5143901,1772582,22595,1.0,0,0,0,"Brand new but filthy. Trash overflowing, dirty...",2021-11-07 23:54:35,"['Brand new but filthy.', 'Trash overflowing, ...",5,"[brand, trash, cleaning, overcharged, return]","[brand new, trash overflowing, counter cleanin..."
2679857,2367318,995712,133866,3.0,8,2,3,Stopped in for dinner and drinks on a Saturday...,2018-03-25 15:30:29,['Stopped in for dinner and drinks on a Saturd...,15,"[dinner, parking, smokers, seating, stand, ban...","[stopped dinner, parking sparse, smokers ruini..."
2679863,6085901,1283829,84658,5.0,1,0,2,"Came here for lunch with my girlfriend, place ...",2018-08-10 04:52:50,"['Came here for lunch with my girlfriend, plac...",4,"[pancakes, , berry, lunch]","[pancakes, , blue berry, fruity drink]"
